In [22]:
import warnings
warnings.filterwarnings("ignore")
import torch
import sys

import torchaudio
import tgt
import os
import numpy as np
from tqdm import tqdm
import librosa
import glob
import soundfile

sys.path.append("../")
from pyfiles.feature_extractor import WavLMExtractor
from vocos.feature_extractors import MelSpectrogramFeatures

sys.path.append("../../cuhksz-phd/sho_util/pyfiles/")
from sound import play_audio

In [23]:
data_dir = "/mntcephfs/lab_data/shoinoue/"
dataset_dir = data_dir + "Dataset/PD-AST/SLT/"
fs = 16000

melgen = MelSpectrogramFeatures(sample_rate=fs)
# wavlm = WavLMExtractor()
speakers = ["Hindi"]
# speakers = [os.path.basename(a) for a in glob.glob(dataset_dir + "*")]
# speakers.sort()

In [43]:
save = True

for spk in speakers:
    print(spk)
    wavlm_dir = f"{dataset_dir}{spk}/wavlm/"
    mel_dir = f"{dataset_dir}{spk}/mel/"
    os.makedirs(wavlm_dir, exist_ok=True)
    os.makedirs(mel_dir, exist_ok=True)
    filenames = glob.glob(dataset_dir + spk + "/wav/*")
    filenames.sort()
    for path in tqdm(filenames):
        # wavlm
        embedding = wavlm.get_feature(path).T.detach().cpu().numpy()
        savepath = wavlm_dir + os.path.basename(path)[:-4] + ".npy"
        if save:
            np.save(savepath, embedding)
            
        # mel
        array = librosa.load(path, 44100)[0]
        soundfile.write("temp.wav", array, 44100)
        y, sr = torchaudio.load("temp.wav")
        # y, sr = torchaudio.load(path)
        if fs!=sr:
            if y.size(0) > 1:  # mix to mono
                y = y.mean(dim=0, keepdim=True)
            y = torchaudio.functional.resample(y, orig_freq=sr, new_freq=fs)
        mel = melgen(y[0])
        savepath = mel_dir + os.path.basename(path)[:-4] + ".npy"
        if save:
            np.save(savepath, mel)

Hindi


100%|██████████| 1132/1132 [04:42<00:00,  4.01it/s]
